# Klasy

In [ ]:
class Gadget:
    def __init__(self, id: int, name: str) -> None:
        self.id = id
        self.name = name

    def use(self) -> None:
        print(f"Using {self.name} with Id#{self.id}")

    def __repr__(self) -> str:
        return f"Gadget(id={self.id}, name={self.name!r})"

    def __str__(self) -> str:
        return f"Gadget - id: {self.id}; name: {self.name}"
    
    def __eq__(self, other):
        return (self.id, self.name) == (other.id, other.name)

In [ ]:
(5).__add__(7)

In [ ]:
g_1 = Gadget(1, "ipad")

In [ ]:
type(g_1)

In [ ]:
id(g_1)

In [ ]:
g_1.name = "ipad 2.0"

In [ ]:
g_1.use()

In [ ]:
print(g_1)

In [ ]:
g_1

In [ ]:
g_2 = eval(repr(g_1))

In [ ]:
g_2

In [ ]:
g_1 == g_2

In [ ]:
g_1.__eq__(g_2)

# Metody `__new__` & `__init__`

In [ ]:
class Dummy:
    def __new__(cls, *args):
        print(f"Dummy.__new__({cls}, {args}) has been called...")
        obj = super().__new__(cls)
        obj.extra_attribute = "EXTRA"
        print(f"Object {obj} has been created...")
        return obj

    def __init__(self, *args) -> None:
        print(f"Dummy.__init__({self}, {args})...")
        self.args = args
        print(f"Object's __dict__: { self.__dict__}")

In [ ]:
d = Dummy(1, "one")

## Kiedy używać `__new__`?

In [ ]:
class UppercaseTuple(tuple):
    def __init__(self, list) -> None:
        print(f"Start changes for {list}")

        for i, item in enumerate(list):
            self[i] = item.upper()

In [ ]:
UppercaseTuple(['one', 'two', 'three'])

In [ ]:
class UppercaseTuple(tuple):
    def __new__(cls, list):
        print(f"Start changes for {list}")
        new_content = [item.upper() for item in list]       
        return super().__new__(cls, new_content)

In [ ]:
UppercaseTuple(['one', 'two', 'three'])

# Metody statyczne i metody klasy

In [ ]:
class CountedObject(object):
    count = 0   # statyczna składowa
    
    def __init__(self):
        CountedObject.count += 1
    
    @staticmethod  # statyczna metoda
    def get_count():
        return CountedObject.count

In [ ]:
lst = [CountedObject() for i in range(10)]

In [ ]:
CountedObject.count

In [ ]:
CountedObject.get_count()

In [ ]:
lst[3].count = 42

In [ ]:
class Person:
    name = "unknown"

In [ ]:
p1 = Person()

In [ ]:
p1.name

In [ ]:
p1.name = "Jan"

In [ ]:
p1.name

In [ ]:
class Date:
    year = 2023

    def __init__(self, day, month, year = None):
        self.day = day
        self.month = month
        if year:
            self.year = year 
    
    @classmethod
    def from_string(cls, date_as_string):        
        day, month, year = date_as_string.split('-')
        return cls(int(day), int(month), int(year)) # utworzenie instancji klasy cls

    @classmethod
    def update_default_year(cls, value):
        cls.year = value

In [ ]:
Date.year

In [ ]:
d1 = Date(25, 9)

In [ ]:
d1.year

In [ ]:
d1.__dict__

In [ ]:
Date.update_default_year(2024)

In [ ]:
d1.year

In [ ]:
d2 = Date.from_string("25-9-2023")

In [ ]:
d2.__dict__

In [ ]:
d1.from_string("3-3-2022")

# Deskryptor

## Non-data descriptor

In [ ]:
import os

class DirectorySize:
    def __get__(self, instance, owner_class):
        print(f'Access to {instance} using descriptor {self}')
        return len(os.listdir(instance.directory_name))
    

class Directory:
    size = DirectorySize() # descriptor instance

    def __init__(self, directory_name):
        self.directory_name = directory_name # regular instance attribute

In [ ]:
local_dir = Directory('.')

In [ ]:
local_dir.__dict__

In [ ]:
local_dir.size

## Data descriptor

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

class LoggedAccess:
    def __set_name__(self, owner, name):
        self.public_name = name
        self.private_name = '_' + name
        logging.info('Setting names: %r and %r', self.public_name, self.private_name)

    def __get__(self, instance, owner_class=None):
        value = getattr(instance, self.private_name)
        logging.info('Accessing %r.%r giving %r', instance, self.public_name, value)
        return value

    def __set__(self, instance, value):
        logging.info('Updating %r.%r to %r', instance, self.public_name, value)
        setattr(instance, self.private_name, value)


class Person:
    age = LoggedAccess()             # Descriptor instance
    name = LoggedAccess()

    def __init__(self, name, age):
        self.name = name                # Regular instance attribute
        self.age = age                  # Calls __set__()

    def birthday(self):
        self.age += 1                   # Calls both __get__() and __set__()

In [ ]:
p1 = Person("Jan", 22)

In [ ]:
p1.age

In [ ]:
p1.age = 44

In [ ]:
p1.age

In [ ]:
p1.name

In [ ]:
p1.birthday()

### ReadOnlyProperty

In [ ]:
class ReadOnlyProperty:
    def __init__(self, fget):
        self.fget = fget

    def __get__(self, instance, owner):
        return self.fget(instance)
    
    def __set__(self, instance, value):
        raise AttributeError("Attribute is read-only")

In [ ]:
class Data:
    def __init__(self, data: int, name: str = "default") -> None:
        self._data = data
        self._name = name

    # def data(self) -> int:
    #    return self._data
    
    # data = ReadOnlyProperty(data)

    @ReadOnlyProperty
    def data(self):
        return self._data
    
    @property
    def name(self):
        return self._name

In [ ]:
data1 = Data(42)

In [ ]:
data1.data

In [ ]:
data1.data = 665

# Slots

In [ ]:
class Pixel:
    __slots__ = ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y

In [ ]:
p1 = Pixel(20, 23)

In [ ]:
p1.__dict__

In [ ]:
p1.x

In [ ]:
p1.y

In [ ]:
class ColorPixel(Pixel):

    __slots__ = ('color')

    def __init__(self, x, y, color):
        super().__init__(x, y)
        self.color = color

In [ ]:
cp1 = ColorPixel(10, 20, 233)

In [ ]:
cp1.__dict__

In [ ]:
cp1.color

# Dziedziczenie

In [ ]:
class A:
    def __init__(self):
        print("A")

class B(A):
    def __init__(self):
        super().__init__()
        print("B")

class C(A):
    def __init__(self):
        super().__init__()
        print("C")

class D(B, C):
    def __init__(self):
        super().__init__()
        print("D")

In [ ]:
D()

In [ ]:
D.__mro__

In [ ]:
class Root:
    
    def ping(self):
        print(f"{self}.ping() in Root")

    def pong(self):
        print(f"{self}.pong() in Root")

    def __repr__(self):
        cls_name = type(self).__name__
        return f"<instance of {cls_name}>"
    

class A(Root):
    
    def ping(self):
        print(f"{self}.ping() in A")
        super().ping()

    def pong(self):
        print(f"{self}.pong() in A")
        return super().pong()
    
class B(Root):
    
    def ping(self):
        print(f"{self}.ping() in B")
        super().ping()

    def pong(self):
        print(f"{self}.pong() in B")
        super().pong()


class Leaf(A, B):
    
    def ping(self):
        print(f"{self}.ping() in Leaf")
        super().ping()


In [ ]:
Leaf.__mro__

In [ ]:
leaf1 = Leaf()

In [ ]:
leaf1.ping()

In [ ]:
class U:
    def ping(self):
        print(f"{self}.ping() in U")
        super().ping()

class LeafUA(U, A):
    def ping(self):
        print(f"{self}.ping() in LeafUA")
        super().ping()

In [ ]:
u = U()

In [ ]:
u.ping()

In [ ]:
leaf2 = LeafUA()

In [ ]:
leaf2.ping()

## Mixins

In [21]:
class ComparableMixin:
    def __ne__(self, other):
        return not (self.__eq__(other))
    
    def __le__(self, other):
        return self < other or self == other
    
    def __gt__(self, other):
        return not self <= other
    
    def __ge__(self, other):
        return self == other or self > other
    

class BaseValue:
    def __init__(self, value):
        self.value = value

class Value(BaseValue, ComparableMixin):
    def __init__(self, value):
        super().__init__(value)

    def __eq__(self, other):
        return self.value == other.value
    
    def __lt__(self, other):
        return self.value < other.value

In [22]:
v1 = Value(10)

In [23]:
v1 == Value(10)

True

In [24]:
v1 != Value(10)

False

In [25]:
v1 >= Value(5)

True

### Mixins w module collections.abc

In [26]:
from collections.abc import Sequence

class VerboseTuple(Sequence):
    """Custom class that is exactly like a tuple but does some
    extra magic.

    Sequence:
    -------------------
    Inherits From: Reversible, Collection
    Abstract Methods: __getitem__, __len__
    Mixin Methods: __contains__, __iter__, __reversed__, index,
            and count
    """

    def __init__(self, *args):
        self.args = args

    @classmethod
    def _classname(cls):
        # This method just returns the name of the class
        return cls.__name__
    
    def __getitem__(self, index):
        print(f"Method: __getitem__, Index: {index}")
        return self.args[index]

    def __len__(self):
        print(f"Method: __len__")
        return len(self.args)

    def __repr__(self):
        return f"{self._classname()}{tuple(self.args)}"

In [28]:
vt = VerboseTuple(1, 2, 3)

In [29]:
vt

VerboseTuple(1, 2, 3)

In [30]:
print(f"Abstract Methods: {set(Sequence.__abstractmethods__)}")
print(f"Mixin Methods: { {k for k, v in Sequence.__dict__.items() if callable(v)} }")

Abstract Methods: {'__len__', '__getitem__'}
Mixin Methods: {'__reversed__', '__contains__', 'count', 'index', '__iter__', '__getitem__'}


In [31]:
vt.index(1)

Method: __getitem__, Index: 0


0

In [32]:
len(vt)

Method: __len__


3

In [33]:
tuple(reversed(vt))

Method: __len__
Method: __getitem__, Index: 2
Method: __getitem__, Index: 1
Method: __getitem__, Index: 0


(3, 2, 1)

# ABC

In [49]:
from abc import ABC, abstractmethod

class IBankAccount(ABC):
    @abstractmethod
    def withdraw(self, amount: float) -> None:
        pass

    @abstractmethod
    def deposit(self, amount: float) -> None:
        pass

    @property
    @abstractmethod
    def balance(self) -> float:
        pass

In [50]:
IBankAccount.__abstractmethods__

frozenset({'balance', 'deposit', 'withdraw'})

In [51]:
class BankAccount(IBankAccount):
    def __init__(self, initial_balance: float) -> None:
        self.__balance = initial_balance
    
    @property
    def balance(self) -> float:
        return self.__balance

    def withdraw(self, amount: float) -> None:
        self.__balance -= amount

    def deposit(self, amount: float) -> None:
        self.__balance += amount


class DebitAccount(BankAccount):
    def __init__(self, initial_balance: float, debit_limit: float) -> None:
        super().__init__(initial_balance)
        self.__debit_limit = debit_limit

    @property
    def debit_limit(self) -> float:
        return self.__debit_limit
    
    @debit_limit.setter
    def debit_limit(self, limit: float) -> None:
        self.__debit_limit = limit

    def withdraw(self, amount: float) -> None:
        if self.balance - amount < -self.debit_limit:
            raise ValueError("Insufficient funds")
        self._BankAccount__balance -= amount

In [52]:
account1 = BankAccount(100.0)
account2 = DebitAccount(1000, 200)

In [53]:
account1.withdraw(150)
account1.balance

-50.0

In [54]:
account2.withdraw(2000)

ValueError: Insufficient funds

In [55]:
def client(account: IBankAccount):
    account.withdraw
    account.withdraw(89)

    print(account.balance)

In [56]:
client(account1)

-139.0


# Protocols

In [57]:
from typing import Protocol


class Stream(Protocol):
    def read(self, data: str): pass
    def write(self, data: str): pass


class StrReaderWriter:
    def read(self, data: str):
        print(f'Reading data: {data}')

    def write(self, data: str):
        print(f'Writing data: {data}')

In [ ]:
def client_io(stream: Stream):
    stream.read("text")
    stream.write("another text")